In [ ]:
!pip install -q transformers accelerate gradio

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"  # Lightweight, fast

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
def generate_code(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.1,
            top_k=40,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id
        )
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Return only the code, remove repeated prompt
    return generated[len(prompt):].strip()


In [ ]:
import gradio as gr

theme = gr.themes.Base().set(
    body_background_fill="#f0f4ff",
    input_background_fill="#ffffff",
    button_primary_background_fill="#4caf50",
    button_primary_text_color="#ffffff",
    # Removed the problematic text_color argument
    block_background_fill="#e6f0fa",
)

with gr.Blocks(theme=theme, title="SnakeScript AI", css=".gr-button {font-size: 16px !important;}") as demo:
    gr.Markdown(
        "<h1 style='text-align:center; color:#1a73e8;'> SnakeScript AI</h1>"
        "<p style='text-align:center;'>Enter an English instruction, and let AI code for you!</p>"
    )

    with gr.Row():
        prompt = gr.Textbox(lines=3, placeholder="e.g., Write Python code to print the Fibonacci sequence.", label="Instruction")

    with gr.Row():
        generate_btn = gr.Button("Your code in process..💡", variant="primary")

    output_code = gr.Code(label="Generated Python Code", language="python")

    generate_btn.click(fn=generate_code, inputs=prompt, outputs=output_code)

demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c296996e03af369a6d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c296996e03af369a6d.gradio.live


In [ ]:
import gradio as gr
import time

# Enhanced theme with better visual hierarchy
theme = gr.themes.Base().set(
    body_background_fill="#f0f4ff",          # Light blue background
    input_background_fill="#ffffff",          # White input areas
    button_primary_background_fill="#4caf50", # Green primary button
    button_primary_text_color="#ffffff",      # White button text
    button_primary_background_fill_hover="#388e3c",  # Darker green on hover
    block_background_fill="#e6f0fa",         # Slightly darker blue for blocks
    block_label_background_fill="#1a73e8",   # Blue label background
    block_label_text_color="#ffffff",        # White label text
    block_title_text_color="#1a73e8",        # Blue title text
    border_color_primary="#1a73e8"           # Blue borders
)

with gr.Blocks(
    theme=theme,
    title="SnakeScript AI",
    css="""
        .gr-button { font-size: 16px !important; }
        .gr-code { max-height: 500px; overflow-y: auto !important; }
        footer { visibility: hidden !important; }
        .gr-textarea { font-family: monospace !important; }
    """
) as demo:
    # Header with improved styling
    gr.Markdown(
        """
        <div style='text-align: center; margin-bottom: 20px;'>
            <h1 style='color: #1a73e8; margin-bottom: 8px; font-weight: 600;'>
                <span style='color: #4caf50;'>Snake</span>Script AI
            </h1>
            <p style='color: #5f6368; font-size: 16px;'>
                Enter an English instruction, and let AI generate Python code for you!
            </p>
        </div>
        """
    )

    with gr.Row():
        prompt = gr.Textbox(
            lines=3,
            placeholder="e.g., Write Python code to print the Fibonacci sequence",
            label="Your Coding Instruction",
            elem_classes=["prompt-box"]
        )

    with gr.Row():
        generate_btn = gr.Button("Generate Code 💡", variant="primary")
        clear_btn = gr.Button("Clear", variant="secondary")

    with gr.Row():
        output_code = gr.Code(
            label="Generated Python Code",
            language="python",
            interactive=True,  # Allows user to edit output
            elem_id="code-output"
        )

    # Add examples for quick testing
    gr.Examples(
        examples=[
            "Write a function to calculate factorial of a number",
            "Create a Python script to download a file from URL",
            "Implement a bubble sort algorithm with comments"
        ],
        inputs=prompt,
        label="Try these examples:",
        examples_per_page=3
    )

    # Interactive functions
    def generate_and_measure(prompt):
        start_time = time.time()
        code = generate_code(prompt)
        elapsed = f"Generated in {time.time() - start_time:.2f}s"
        return code, elapsed

    # Button actions
    generate_btn.click(
        fn=generate_and_measure,
        inputs=prompt,
        outputs=[output_code, generate_btn]
    )

    clear_btn.click(
        lambda: [None, "Generate Code 💡"],
        outputs=[output_code, generate_btn]
    )

demo.launch(
    debug=True,
    server_port=7860,
    favicon_path="https://www.python.org/static/favicon.ico"
)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d7fbbf65667069f58d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d7fbbf65667069f58d.gradio.live


In [ ]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Initialize FastAPI
app = FastAPI(
    title="SnakeScript AI",
    description="Generate Python code from English instructions using DeepSeek-Coder",
    version="1.0.0"
)

# Load model and tokenizer (auto GPU/CPU detection)
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

# Input schema
class CodeRequest(BaseModel):
    instruction: str

# Output schema
class CodeResponse(BaseModel):
    generated_code: str

# Main API route
@app.post("/generate", response_model=CodeResponse)
async def generate_code(request: CodeRequest):
    prompt = request.instruction.strip()

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.1,
            top_k=40,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id
        )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_code = full_output[len(prompt):].strip()

    return CodeResponse(generated_code=generated_code)

# Health check
@app.get("/")
async def root():
    return {"message": "SnakeScript AI is running!"}


ModuleNotFoundError: No module named 'fastapi'

In [ ]:
!pip install -q fastapi uvicorn # Install fastapi and uvicorn
from fastapi import FastAPI, Request
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Initialize FastAPI
app = FastAPI(
    title="SnakeScript AI",
    description="Generate Python code from English instructions using DeepSeek-Coder",
    version="1.0.0"
)

# Load model and tokenizer (auto GPU/CPU detection)
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

# Input schema
class CodeRequest(BaseModel):
    instruction: str

# Output schema
class CodeResponse(BaseModel):
    generated_code: str

# Main API route
@app.post("/generate", response_model=CodeResponse)
async def generate_code(request: CodeRequest):
    prompt = request.instruction.strip()

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.1,
            top_k=40,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id
        )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_code = full_output[len(prompt):].strip()

    return CodeResponse(generated_code=generated_code)

# Health check
@app.get("/")
async def root():
    return {"message": "SnakeScript AI is running!"}

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]